In [1]:
# ── put this in the first cell of the notebook ──────────────────────
import sys, pathlib

# /Users/…/GaltonBoard/notebooks/Exploratory/  ← cwd when you open the nb
PROJECT_ROOT = pathlib.Path.cwd().resolve().parent.parent   # climb 2 levels
SRC = PROJECT_ROOT / "src"

if str(SRC) not in sys.path:
    sys.path.append(str(SRC))



In [ ]:

# imports + helpers
#######################################################################
import pennylane as qml
from pennylane import numpy as np

import sys, pathlib

# /Users/…/GaltonBoard/notebooks/Exploratory/
PROJECT_ROOT = pathlib.Path.cwd().resolve().parent.parent
SRC = PROJECT_ROOT / "src"

if str(SRC) not in sys.path:
    sys.path.append(str(SRC))

import importlib, circuit.quantum_galton_board as qgb
importlib.reload(qgb)

from utils.misc import triangular_number

np.random.seed(42)

# 1) BUILD **BASE** GALTON BOARD (levels = 8 or else my laptop die)

levels = 8

board_core = qgb.build_galton_circuit(
    levels        = levels,
    num_shots     = None,
    bias          = 0.3,
    coherence = True
)

_ = board_core()
print("max wire in board_core:", max(board_core.device.wires))
# ---- run it once to create a tape ----
                      # analytic, fast, no side‑effects

# now the attribute exists, regardless of PL version
tape = qc.qtape if hasattr(qc, "qtape") else qc.tape
print("number of measurements:", len(tape.measurements))

print("Base wires:", board_core.device.wires.tolist())   # 0..15


# 2) one qaoa layer (p=1); target will be some cost Hamiltonian
# We treat the *right half* (wires 8..15) as a MaxCut graph on 8 nodes
data_wires   = list(range(levels, 2*levels))   # [8..15]
gamma, beta  = qml.numpy.array([0.8], requires_grad=True), qml.numpy.array([0.4], requires_grad=True)

def maxcut_cost(wires):
    H = qml.Hamiltonian([], [])
    for i in range(len(wires)):
        for j in range(i+1, len(wires)):
            H += (0.5 * (qml.Identity(0) - qml.PauliZ(wires[i]) @ qml.PauliZ(wires[j])))
    return H

cost_H = maxcut_cost(data_wires)

def qaoa_layer():
    # phase-separation
    for term, coeff in zip(cost_H.ops, cost_H.coeffs):
        qml.expval(qml.Identity(0))  # placeholder to create tape context
    qml.templates.ApproxTimeEvolution(cost_H, gamma, 1)

    # mixing layer on data qubits
    for w in data_wires:
        qml.RX(2 * beta, wires=w)

# 3) build entire circuit (board + QAOA layer)

dev_full = qml.device("lightning.qubit", wires=2*levels)

@qml.qnode(dev_full)
def full_circuit():
    board_core()          # embeds the biased Galton board
    qaoa_layer()          # one QAOA block on data wires
    return qml.expval(cost_H)

full_cost = full_circuit()
print("Full‑circuit ⟨H⟩:", full_cost)


# 4) **CUT** THE CIRCUIT ALONG THE HALF‑WAY BARRIER  (wires 7|8)

cut_wires = [7]    # edge between wire7 and wire8 splits the device
cut_fragments, fn_reconstruct = qml.transforms.cut_circuit(cut_wires)(full_circuit, explicit=True)

print("Fragments:", len(cut_fragments))
for i, f in enumerate(cut_fragments):
    print(f" ├─ fragment {i} uses wires {f.device.wires.tolist()}")

# Evaluate each fragment on its own (parallelisable)
fragment_vals = [frag() for frag in cut_fragments]

# Re‑assemble the global expectation value
cut_cost = fn_reconstruct(fragment_vals)
print("Cut‑reconstructed ⟨H⟩:", cut_cost)


# Optimise a single QAOA step on the cut version

opt = qml.GradientDescentOptimizer(stepsize=0.2)

params = np.array([0.5, 0.3], requires_grad=True)   # [gamma, beta]

def cut_cost_fn(params):
    g, b = params
    gamma[...] = g; beta[...] = b
    return fn_reconstruct([frag() for frag in cut_fragments])

for it in range(20):
    params = opt.step(cut_cost_fn, params)
    if it % 5 == 0:
        print(f"Iter {it:02d}  cost = {cut_cost_fn(params):.6f}")

print("Optimised params (γ, β):", params)
print("Final cut cost:", cut_cost_fn(params))


WireError: Cannot run circuit(s) on lightning.qubit as they contain wires not found on the device: {16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36}